# Instructions
This notebook allows you to load and evaluate a huggingface model on a subset of BLiMP (a linguistic acceptability judgment dataset) and GLUE (a natural language understanding benchmark collection). It is HIGHLY recommended to clone the GitHub repository and evaluate your model in the command-line; this will give you more freedom in the kinds of models you can evaluate. However, Colab provides a GPU that will allow you to load and evaluate smaller models.

To use this notebook:

1. Start by making a copy of this notebook so that you can make edits and run the code: File > Save a copy in Drive.

2. Set Runtime > Change runtime type > Hardware accelerator to GPU if it isn't already.

3. Run the setup script to install the required packages for evaluating.

4. Upload your model to the colab in the `/content/model_folder/` directory. This folder should include the following files, and probably a couple more depending on the type of model and tokenizer you use:
* `config.json`
* `pytorch_model.bin`
* `tokenizer_config.json`
* `vocab.json`

  a. To obtain these files given your pre-trained model and your tokenizer, load them using huggingface `transformers` and save them using these commands:
```
tokenizer.save_pretrained("./model_dir")
model.save_pretrained("./model_dir")
```
  b. Then, upload all the contents of `model_dir` (including any other files not mentioned above) to the `model_folder` folder in this Colab.

5. Choose the proper model type in the dropdown in the "load model and evaluate" cell. Use "decoder" for autoregressive (sometimes called "causal") language models, like GPT/OPT; "encoder" for masked language models, like BERT/RoBERTa; or "encoder-decoder" for text-to-text models, like T5/BART.

6. Run the cells below to load and evaluate your model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline" "/content/"

In [ ]:
!mkdir model_folder
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/config.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/merges.txt" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/model.safetensors" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/special_tokens_map.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/tokenizer.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/tokenizer_config.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/training_args.bin" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/test_models/model_4_folder/vocab.json" "/content/model_folder"

In [ ]:
!cp -r "/content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/lm_eval" "/content/"

In [ ]:
#@title Setup script { display-mode: "form" }
#@markdown Run this cell to install the necessary packages (may take a few minutes).
%%shell
# Remove previous installation if it exists
cd /content
# mkdir model_folder
pip uninstall -y lm-eval
# rm -rf evaluation-pipeline/

# Install evaluation-pipeline
# cp -r /content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline /content/
cd evaluation-pipeline/
pip install -e ".[colab]"
# Install other necessary packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# Unpack dataset
# unzip filter_data.zip

mkdir: cannot create directory ‘model_folder’: File exists
Obtaining file:///content/evaluation-pipeline
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/accelerate (to revision main) to /tmp/pip-install-4_odshay/accelerate_894968b5d87547bebb4316a52e08271c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-install-4_odshay/accelerate_894968b5d87547bebb4316a52e08271c
  Resolved https://github.com/huggingface/accelerate to commit 650b6659c0856e0a995d04e491b9672bbf49c720
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 120.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of lm-eval to determine which version is compatible with 

In [ ]:
!pip install sacrebleu

In [ ]:
!pip install datasets

In [ ]:
!pip install rouge_score

In [ ]:
!pip install lm_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Evaluation

In [ ]:
#@title Load model and evaluate (BLiMP) { display-mode: "form" }
model = "/content/model_folder/" #@param {"type": "string"}
model_type = "encoder" #@param ["decoder", "encoder", "encoder-decoder"]
# file_name = "examples3.csv" #@param {"type": "string"}
# model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}

%cd /content/evaluation-pipeline
%run /content/evaluation-pipeline/babylm_eval.py \
  "$model" \
  "$model_type" \
  -t "blimp"

/content/evaluation-pipeline


Generating train split: 0 examples [00:00, ? examples/s]


» Assigning unique IDs to 'blimp_from_file+null' docs


INFO:lm_eval.evaluator:
» Assigning unique IDs to 'blimp_from_file+null' docs


Map:   0%|          | 0/1956 [00:00<?, ? examples/s]

...


» Constructing 'blimp_from_file+null' contexts and requests


INFO:lm_eval.evaluator:
» Constructing 'blimp_from_file+null' contexts and requests
100%|██████████| 5535/5535 [00:00<00:00, 6648.26it/s]


» Running all `loglikelihood` requests



INFO:lm_eval.evaluator:
» Running all `loglikelihood` requests
100%|██████████| 11070/11070 [01:01<00:00, 181.36it/s]


subject_verb_agreement:	51.65%

Scores:
anaphor_agreement:	67.08%
argument_structure:	57.66%
binding:	59.10%
control_raising:	56.87%
determiner_noun_agreement:	58.99%
ellipsis:	26.67%
filler_gap:	61.55%
irregular_forms:	74.55%
island_effects:	40.62%
npi_licensing:	47.28%
quantifiers:	54.22%
subject_verb_agreement:	51.65%
